
# Climatology Average
This script illustrates the following concepts:

- Usage of geocat-comp's [climatology_average](https://geocat-comp.readthedocs.io/en/latest/user_api/generated/geocat.comp.climatologies.climatology_average.html#geocat.comp.climatologies.climatology_average) function
- Usage of geocat-datafiles for accessing NetCDF files
- Creating a figure with stacked subplots

## Import packages

Dependencies:

- geocat.comp
- [geocat.datafiles](https://github.com/NCAR/geocat-datafiles) (for accessing data file only)
- [cftime](https://unidata.github.io/cftime/) (installed with geocat.comp)
- [matplotlib](https://matplotlib.org/) (installed with geocat.viz)
- [xarray](https://docs.xarray.dev/en/stable/) (installed with geocat.comp)

In [ ]:
import cftime
import matplotlib.pyplot as plt
import xarray as xr

from geocat.comp import climatology_average
import geocat.datafiles as gdf

## Read in data

We will get the data from the [geocat-datafiles](https://github.com/NCAR/geocat-datafiles) package. This package contains example data used in many of the examples for geocat packages.

Then, we use xarray's [open_dataset](http://xarray.pydata.org/en/stable/generated/xarray.open_dataset.html) function to read the data into an xarray dataset, choose a single model from the ensemble run, and extract its surface temperature data into `temp`

In [ ]:
ds = xr.open_dataset(gdf.get('netcdf_files/atm.20C.hourly6-1990-1995-TS.nc'))
ds = ds.isel(member_id=0)  # select one model from the ensemble

temp = ds.TS  # surface temperature data

## Calculate daily and monthly climate averages using `climatology_average`


Next, we use `geocat.comp.climatologies.climatology_average` to calculate averages across all years in a given dataset.

Note that while daily and monthly climatology averages are demonstrated here, you could also use the frequency keyword argments `hour` or `season` to calculate hourly or seasonal climatology averages.

In [ ]:
daily = climatology_average(temp, 'day')
monthly = climatology_average(temp, 'month')

# Convert datetimes to number of hours since 1990-01-01 00:00:00
# This must be done in order to use the time for the x axis
time_num_raw = cftime.date2num(temp.time, 'hours since 1990-01-01 00:00:00')
time_num_day = cftime.date2num(daily.time, 'hours since 1990-01-01 00:00:00')
time_num_month = cftime.date2num(monthly.time,
                                 'hours since 1990-01-01 00:00:00')

# Start and end time for axes limit in units of hours since 1990-01-01 00:00:00
tstart = time_num_raw[0]
tend = time_num_raw[-1]

## Plot

Next we'll create a simple plot to display our climatology data.

The top subplot is raw surface temperature data from a model run with a
temporal resolution of 6-hours.

The middle subplot shows the output of the raw data being aggregated using the
`climatology_average` function with the `freq` argument set to 'daily'. This
function with that setting finds the average daily temperature for each day of
the year. The output has adjusted datetimes instead of using integers to denote
the day of the year for the time axis. The year for the outputted data is the
floor of the median year of the inputted data, which is 1993 in this case.

The bottom subplot shows the output of `climatology_average` with the `freq`
argument set to `monthly`. This works much the same as for the middle plot;
however, the data is now grouped by month which yeilds a smoother curve. The
time axis is adjusted in the same way, except now there are only 12 data points
with one for each month.

In [ ]:
fig, ax = plt.subplots(3,
                       1,
                       figsize=(8, 10),
                       sharex=True,
                       sharey=True,
                       constrained_layout=True)

# Plot data
ax[0].plot(time_num_raw, temp.data)
ax[0].set_ylabel('Raw Data')

ax[1].plot(time_num_day, daily.data)
ax[1].set_ylabel('Daily Climatology')

ax[2].plot(time_num_month, monthly.data)
ax[2].set_ylabel('Monthly Climatology')

ax.set_xlabel('Time')

# Add title manually to control spacing
title_str = 'Climatology Average on 6-hourly Data \n ' + temp.long_name + ' (' + temp.units + ')'
fig.suptitle(title_str, fontsize = 20)

plt.show();